# SEMA-3D

**SEMA-3D** is a fine-tuned ESM-IF1 model aimed to predict epitope resiudes based on therity structures

The MIT License (MIT)

Copyright (c) 2016 AYLIEN

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

### Finetuning ESM-IF1 for epitope prediction tasks

In [13]:
import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "../torch_hub"
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [14]:
import scipy
import sklearn
import esm
import pickle

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch import nn
import math

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction
from transformers import EsmTokenizer, EsmForMaskedLM

from esm.pretrained import load_model_and_alphabet_hub
from esm.inverse_folding.util import CoordBatchConverter

from sklearn.metrics import r2_score, mean_squared_error

from biotite.structure.residues import get_residues

from utils.foldseek_util import get_struc_seq
from pathlib import Path

In [15]:
def esmStructDataset(pdb_path, chain):
    '''
    Convert PDB-file into dataset format
    
        Parameters:
            pdb_path (Path): path to pdb-file
        Returns:
            dict (dict): dictionary, where keys are properties of the protein's tertiary structure
    '''
    
    try:
        parsed_seqs = get_struc_seq("../bin/foldseek", pdb_path)[chain.upper()]
    except:
        parsed_seqs = get_struc_seq("../bin/foldseek", pdb_path)[chain.lower()]
    seq, foldseek_seq, combined_seq = parsed_seqs

    try:
        structure = esm.inverse_folding.util.load_structure(str(pdb_path), chain.upper())
    except:
        structure = esm.inverse_folding.util.load_structure(str(pdb_path), chain.lower())
        
    resi_index = get_residues(structure)[0]
    resi_aa    = get_residues(structure)[1]
    resi_keys     = []
    for resi_index_,resi_aa_ in zip(get_residues(structure)[0],get_residues(structure)[1]):
        key = (str(resi_aa_),resi_index_)
        resi_keys.append(key)    
    # print('pdb_path', pdb_path)
    # print('resi_keys', len(resi_keys), resi_keys)
    # print('seq', len(seq), seq)
    assert len(resi_keys) == len(seq), 'foldseek and esm.inverse_folding.util.load_structure returns different seqs'

    return {"seq":seq,
            "foldseek_seq":foldseek_seq,
            "combined_seq":combined_seq,
            "residues": resi_keys
            }

In [16]:
def create_Dataset(path=''):
    '''
    Create dataset of protein's tertiary structure or load it form pickle-file
        Parameters:
            path (Path): path to pikle object with dataset of protein's tertiary structure
        Returens:
            esm_structs (dict): dataset of protein's tertiary structure
    '''  
    path_pdbs = Path("/mnt/nfs_protein/ivanisenko/SEMA/dataset/3D/").glob("*.pdb")

    esm_structs = {}
    for pdb_path in list(path_pdbs):
        chain = str(pdb_path).split('.')[0][-1]
        tmp_dict = esmStructDataset(pdb_path, chain)

        esm_structs[pdb_path.name.split(".pdb")[0]] = tmp_dict
        
    return esm_structs    

In [17]:
def prepareEsmDataset(dataset, structs_data):
    dataset_esm = []
    bad_prots = []
    for k in dataset:
        entity = dataset[k]
        pdb_id = entity["pdb_id_chain"]
        assert pdb_id in structs_data
        struct_data = structs_data[pdb_id]
        
        if len(np.unique(entity['contact_number'])) == 1:
            bad_prots.append(pdb_id)
            continue

        # struct_data["cn"] = [i for i in entity["contact_number"] if i != -100]
        struct_data["cn"] = [-100]* len(struct_data['seq'])
        
        key_map = {(key[0],key[1]):i for i, key in enumerate(zip(entity['resi_name'],
                                                                 entity['resi_pos']))
                  } 
    
        for i,key in enumerate(struct_data["residues"]):
            key = (key[0],key[1])
            if key not in key_map:
                struct_data["cn"][i] = -100
                continue
            struct_data["cn"][i]= entity['contact_number'][key_map[key]]
            # struct_data["binary"][i]= entity['contact_number_binary'][key_map[key]]

        assert len([i for i in struct_data["cn"] if i != -100]) > 0, print(pdb_id + '\n', key_map,'\n', struct_data["residues"], '\n', struct_data["cn"])
        
        if len(struct_data["seq"])>1500:
            print("Skip long  ", pdb_id, len(struct_data["seq"]))
            continue
        dataset_esm.append(struct_data)

    print('Number of proteins with invalid contact_number: ', {len(bad_prots)})
    
    dataset_esm = pd.DataFrame(dataset_esm)[['combined_seq', 'cn']]

    return PDB_Dataset(dataset_esm)

In [18]:
class PDB_Dataset(Dataset):
    """
    A class to represent a sutable data set for model. 
    
    convert original pandas data frame to model set,
    where 'token ids' is ESM-1v embedings corresponed to protein sequence (max length 1024 AA)
    and 'lables' is a contact number values
    Attributes:
        df (pandas.DataFrame): dataframe with two columns: 
                0 -- preotein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- contcat number values in list [0, 0.123, 0.23, -100, 1.34] format
        esm1v_batch_converter (function):
                    ESM function callable to convert an unprocessed (labels + strings) batch to a
                    processed (labels + tensor) batch.
        label_type (str):
                type of model: regression or binary

    """
    def __init__(self, df, label_type ='regression'):
        """
        Construct all the necessary attributes to the PDB_Database object.
        
        Parameters:
            df (pandas.DataFrame): dataframe with two columns: 
                0 -- protein sequence in string ('GLVM') or list (['G', 'L', 'V', 'M']) format
                1 -- contcat number values in list [0, 0.123, 0.23, -100, 1.34] format
            label_type (str):
                type of model: regression or binary
        """
        model_path = "westlake-repl/SaProt_650M_PDB"
        self.tokenizer = EsmTokenizer.from_pretrained(model_path)

        self.df = df
        self.label_type = label_type

    def __getitem__(self, idx):
        item = {}
        
        # item['token_ids'] = self.tokenizer.tokenize(self.df.iloc[idx, 0][:1022])
        item['token_ids'] = self.tokenizer(self.df.iloc[idx, 0][:2044], padding = False, return_tensors="pt")['input_ids']
        item['labels'] = torch.unsqueeze(torch.tensor(self.df.iloc[idx, 1][:1022]),0)
        # item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1][:1022]),0)

        # print('PDB_Dataset pdb_id', self.df.iloc[idx, 2][:2048])
        # print('PDB_Dataset seq', len(self.df.iloc[idx, 0][:2048]), self.df.iloc[idx, 0][:2048])
        # print('PDB_Dataset token_ids', item['token_ids'].shape)
        # print('PDB_Dataset labels', item['labels'].shape)
        return item

    def __len__(self):
        return len(self.df)

In [19]:
class SaProtForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2):
        super().__init__()
        self.num_labels = num_labels
        model_path = "westlake-repl/SaProt_650M_PDB"
        self.encoder = EsmForMaskedLM.from_pretrained(model_path)

        self.classifier = nn.Linear(446, self.num_labels)

    def forward(self, token_ids, labels):
        # print('SaProtForTokenClassification token_ids', token_ids.shape)
        outputs = self.encoder(input_ids = token_ids)['logits']
        # print('SaProtForTokenClassification encoder outputs', outputs.shape)
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)
        # print('logits', logits.shape)
        # print('logits', logits)
        return SequenceClassifierOutput(logits=logits)

In [20]:
def compute_metrics_regr(p: EvalPrediction):
    
    preds = p.predictions[:,:,1]

    batch_size, seq_len = preds.shape    
    out_labels, out_preds = [], []

    for i in range(batch_size):
        for j in range(seq_len):
            if p.label_ids[i, j] > -1:
                out_labels.append(p.label_ids[i][j])
                out_preds.append(preds[i][j])
                
    # out_labels_regr = [math.log(t+1) for t in out_labels]
    out_labels_regr = out_labels

    
    return {
        "pearson_r": scipy.stats.pearsonr(out_labels_regr, out_preds)[0],
        "mse": mean_squared_error(out_labels_regr, out_preds),
        "r2_score": r2_score(out_labels_regr, out_preds)
    }


In [21]:
def model_init():
    return SaProtForTokenClassification().cuda()

In [22]:
class MaskedMSELoss(torch.nn.Module):
    def __init__(self):
        super(MaskedMSELoss, self).__init__()

    def forward(self, inputs, target, mask):   
        # print('loss inputs',inputs.shape, inputs)
        # print('target',target.shape, target)
        # print('mask',mask.shape)
        diff2 = (torch.flatten(inputs[:,:,1]) - torch.flatten(target)) ** 2.0 * torch.flatten(mask)
        result = torch.sum(diff2) / torch.sum(mask)
        if torch.sum(mask)==0:
            return torch.sum(diff2)
        else:
            #print('loss:', result)
            return result

    
class MaskedRegressTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):

        # print('trainer inputs',inputs['token_ids'].shape, inputs['token_ids'])
        # print('trainer labels',inputs['labels'].shape, inputs['labels'])
        # # inputs = inputs[0]
        # # labels = inputs.pop("labels")
        labels = inputs["labels"]
        labels = labels.squeeze().detach().cpu().numpy().tolist()
        # labels = [math.log(t+1) if t!=-100 else -100 for t in labels]
        labels = torch.unsqueeze(torch.FloatTensor(labels), 0).cuda()
        masks = ~torch.eq(labels, -100).cuda()
        
        #masks = inputs.pop("masks")
        outputs = model(**inputs)
        logits = outputs.logits

        loss_fn = MaskedMSELoss()
        loss = loss_fn(logits, labels, masks)
        
        return (loss, outputs) if return_outputs else loss        
        
def collator_fn(x):
    if len(x)==1:
        return x[0]
    # print('x:', x)
    return x


## Data

In [10]:
esm_structs = create_Dataset(path='')

In [23]:
print('Number of structures:', len(esm_structs.keys()))

Number of structures: 1377


In [24]:
import json
path = '/mnt/nfs_protein/ivanisenko/SEMA/dataset/SEMA_DS.json'
def load_datasets(path, filtr_length = False):
    with open(path, 'r') as file:
        dataset = json.load(file)
        dataset = pd.DataFrame(dataset)

    dataset['pdb_id_chain'] = dataset['pdb_path'].str.split('/', expand = True)[2].str.split('.', expand = True)[0]
    
    if filtr_length:
            dataset = dataset[dataset['seq'].str.len() >= filtr_length]

    to_residue_name = lambda x: [i['residue_name'] if i != None else i for i in x]
    to_residue_pos = lambda x: [i['residue_number'] if i != None else i for i in x]
    dataset.loc[:, 'resi_name'] = dataset['residues'].apply(to_residue_name)
    dataset.loc[:, 'resi_pos'] = dataset['residues'].apply(to_residue_pos)

    train_set = dataset[dataset['ds_type'] == 'train']
    # test_set = dataset[dataset['ds_type'] == 'test_new']
    test_set = dataset[dataset['ds_type'] == 'test_original']

    # train_set = train_set.iloc[0:10, :]
    # test_set = test_set.iloc[0:10, :]

    train_set = train_set.reset_index().to_dict(orient='index')
    test_set = test_set.reset_index().to_dict(orient='index')

    return train_set , test_set

train_set , test_set = load_datasets(path, filtr_length= False)

train_ds = prepareEsmDataset(train_set, esm_structs)
test_ds = prepareEsmDataset(test_set, esm_structs)

Skip long   5I5K_A 1632
Number of proteins with invalid contact_number:  {21}
Number of proteins with invalid contact_number:  {0}


## Model training and test 

In [25]:
training_args = TrainingArguments(
    output_dir='./results_fold' ,          # output directory
    num_train_epochs=2,          # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=20,                # number of warmup steps for learning rate scheduler
    learning_rate=1e-05,             # learning rate
    weight_decay=0.0,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    save_strategy = "no",
    do_train=True,                   # Perform training
    do_eval=False,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=1,  # total number of steps before back propagation
    fp16=False,                       # Use mixed precision
    run_name="PDB_regr",      # experiment name
    seed=42,                         # Seed for experiment reproducibility
    load_best_model_at_end=False,
    metric_for_best_model="eval_r2",
    greater_is_better=True,

)

In [26]:
#create direactory to weights storage
if not os.path.exists("../models/"):
    os.makedirs("../models/")

In [27]:
torch.cuda.empty_cache()

In [17]:
# torch.cuda.empty_cache()
trainer = MaskedRegressTrainer(
    model=model_init(),                 # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset = train_ds,    # training dataset
    eval_dataset  = test_ds,    # evaluation dataset
    data_collator = collator_fn,
    compute_metrics = compute_metrics_regr
)
trainer.train()

#save weights
torch.save(trainer.model.state_dict(), "../models/sema_saprot_continous_noncut_log10_0.pth")

Some weights of EsmForMaskedLM were not initialized from the model checkpoint at westlake-repl/SaProt_650M_PDB and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight', 'esm.embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Pearson R,Mse,R2 Score
1,0.138800,0.144210,0.361670,0.141821,0.110053
2,0.114700,0.140838,0.365422,0.140649,0.117408


# Multimer test

In [19]:
from biopandas.pdb import PandasPdb
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from IPython.display import clear_output


aa_3_to_1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [20]:
from tqdm import tqdm 
import numpy as np

def esmStructDataset(pdb_path, chain):
    '''
    Convert PDB-file into dataset format
    
        Parameters:
            pdb_path (Path): path to pdb-file
        Returns:
            dict (dict): dictionary, where keys are properties of the protein's tertiary structure
    '''
    
    try:
        parsed_seqs = get_struc_seq("bin/foldseek", pdb_path)[chain.upper()]
    except:
        parsed_seqs = get_struc_seq("bin/foldseek", pdb_path)[chain.lower()]
    seq, foldseek_seq, combined_seq = parsed_seqs

    return {"seq":seq,
            "foldseek_seq":foldseek_seq,
            "combined_seq":combined_seq
            }

def prepare_multimer_test_1D_model(path_to_pdbs, gap_length = 0):
    
    path_pdbs = Path(path_to_pdbs).glob("*.pdb")
    one_chain_prots = []
    test_df = pd.DataFrame(columns = ['seq', 'concat_number'])

    for pdb_path in tqdm(list(path_pdbs)):
        # read and savepdb
        pdb_df =  PandasPdb().read_pdb(str(pdb_path))
        pdb_df.to_pdb('tmp_raw_pdb.pdb')
        # fix insertions in residue number (tool convert int+ letter (ex 150A) value to int (ex 151))
        _ = os.system("pdb_delinsertion tmp_raw_pdb.pdb > tmp_raw_pdb_fix_ins.pdb")
        
        # read pdb, change all chains to M and save to tmp_M.pdb
        pdb_df =  PandasPdb().read_pdb('tmp_raw_pdb_fix_ins.pdb')
        atom_df =  pdb_df.df['ATOM'].copy()
        atom_df.loc[:, 'chain_id'] = 'M'
        pdb_df.df['ATOM'] = atom_df
        pdb_df.to_pdb('tmp_M.pdb')
        # change residue_number in tmp_M.pdb
        _ = os.system("pdb_reres -1 tmp_M.pdb > tmpReind_M.pdb")

        multimer_seq = esmStructDataset('tmpReind_M.pdb', 'M')['combined_seq']
        
        pdb_df =  PandasPdb().read_pdb(str('tmpReind_M.pdb'))
        atom_df =  pdb_df.df['ATOM'].copy()
        # change mask values from -1 to -100
        atom_df.loc[atom_df['b_factor'] == -1, 'b_factor'] = -100
        # make nonmask values - log10
        atom_df.loc[atom_df['b_factor'] >= 0, 'b_factor'] = atom_df.loc[atom_df['b_factor'] >= 0, 'b_factor'].apply(lambda x: np.log10(x + 1))
        
        multimer_contacts = atom_df[atom_df.atom_name == 'CA']['b_factor'].tolist()

        assert len(multimer_contacts) == len(multimer_seq)/2, print(str(pdb_path) + '\n', f'cn: {len(multimer_contacts)}{multimer_contacts},\ncombined seq: {len(multimer_seq)}, {multimer_seq}')
            
        test_df.loc[test_df.shape[0] +1, :] = {'seq': multimer_seq, 'concat_number': multimer_contacts}
        clear_output()
    
    print('PDBs with only one chain:', len(one_chain_prots))

    return test_df

In [22]:
# test_df = prepare_multimer_test_1D_model('/mnt/nfs_protein/ivanisenko/SEMA/dataset/multimers/', gap_length = 0)
multi_test_ds = PDB_Dataset(test_df)

In [68]:
trainer.evaluate(multi_test_ds)

{'eval_loss': 0.12266069650650024,
 'eval_pearson_r': 0.4212938220320703,
 'eval_mse': 0.11615478992462158,
 'eval_r2_score': 0.17550282305813802,
 'eval_runtime': 87.2102,
 'eval_samples_per_second': 8.864,
 'eval_steps_per_second': 8.864,
 'epoch': 2.0}

# Calculate threshold

In [30]:
from sklearn.metrics import roc_curve
from tqdm import tqdm

path = '/home/shevtsov/SEMAi/models/sema_saprot_continous_noncut_log10_0.pth'
model=model_init()
model.load_state_dict(torch.load(path))
model.eval()
model.cuda()

with torch.no_grad():
    preds=[]
    labels = []
    for it in tqdm(test_ds):
        preds.append(model.forward(it['token_ids'].cuda(), labels = None)[0][0][:,1].cpu().numpy())
        labels.append(it['labels'].squeeze(0).numpy())

  0%|          | 0/72 [00:00<?, ?it/s]

100%|██████████| 72/72 [00:19<00:00,  3.66it/s]


ValueError: unknown format is not supported

In [35]:
np.concatenate(preds)
fpr, tpr, thresholds = roc_curve(np.concatenate(preds), np.concatenate(labels), pos_label=2)

ValueError: continuous format is not supported

In [36]:
np.concatenate(preds)

array([0.19790691, 0.17728502, 0.12358756, ..., 0.40309232, 0.42081374,
       0.40595084], dtype=float32)